In [1]:
import re
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import datetime
import arrow
from ics import Calendar, Event

In [2]:
# 学籍番号
student_id = "*******"

# パスワード
password = "*******"

In [3]:
# chrome driverの設定
options = ChromeOptions()
options.add_argument("--headless")
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(
    options=options,
    service=ChromeService(ChromeDriverManager().install())
    )

# urlを指定
url="https://muscat.musashino-u.ac.jp/portal/top.do"
driver.get(url)

# 学籍番号を入力
login = "s" + str(student_id)
driver.find_element(By.XPATH,"//*[@id='userId']").send_keys(login)

# パスワードを入力
driver.find_element(By.XPATH,"//*[@id='password']").send_keys(password)

# ログインボタンをクリック
btn = driver.find_element(By.XPATH,"//*[@id='loginButton']")
btn.click()

# メニューの非表示を解除
menuList = driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div[6]/ul/li[3]/ul")
driver.execute_script("arguments[0].setAttribute('style','display: block;')", menuList)

# 履修登録確認にアクセス
myCalendar = driver.find_element(By.XPATH,"/html/body/div[1]/div[1]/div[6]/ul/li[3]/ul/li[1]/a")
myCalendar.send_keys(Keys.ENTER)

# 時間割を抽出
table = []
for j in range(3, 16, 2):
    for i in range(3, 14, 2):
        coma_xpath = f"/html/body/div/div[2]/table/tbody/tr/td[1]/form/div[2]/div[3]/table/tbody/tr[{j}]/td[{i}]/table/tbody/tr/td"
        coma = driver.find_element(By.XPATH, coma_xpath)
        coma_text = coma.get_attribute("innerHTML")
        pattern = r'<br>(.*?)<br>'
        matches = re.findall(pattern, coma_text, re.DOTALL)
        table.append(matches[0].strip())
jikanwari = [table[0:6], table[6:12], table[12:18], table[18:24], table[24:30], table[30:36], table[36:42]]

# chromeを終了
driver.close()

In [4]:
jikanwari

[['', 'データマイニング', '', '', '', ''],
 ['', '', '', '', '', ''],
 ['ビジネスモデル創出', '', '専門コース演習Ⅲ（ソーシャルイノベーション）', '', '卒業論文創成課題', ''],
 ['ビジネスモデル創出', '', '専門コース演習Ⅲ（ソーシャルイノベーション）', '', '卒業論文創成課題', ''],
 ['', '', '', '仏教（生き方を考える）発展[a11]', '', ''],
 ['', '', '', '', '', ''],
 ['', '', '', '', '', '']]

In [5]:
# 学期
semester = {
    0: [datetime.date(2023,4,15), datetime.date(2023,6,10)],
    1: [datetime.date(2023,6,15), datetime.date(2023,8,2)],
    2: [datetime.date(2023,9,22), datetime.date(2023,11,13)],
    3: [datetime.date(2023,11,21), datetime.date(2024,1,27)],
}

# 休日
holiday = [
    datetime.date(2023,4,29), datetime.date(2023,5,1), datetime.date(2023,5,2),
    datetime.date(2023,5,3), datetime.date(2023,5,4), datetime.date(2023,5,5),
    datetime.date(2023,5,20), datetime.date(2023,10,6), datetime.date(2023,10,9),
    datetime.date(2023,12,26), datetime.date(2023,12,27), datetime.date(2023,12,28),
    datetime.date(2023,12,29), datetime.date(2023,12,30), datetime.date(2024,1,1),
    datetime.date(2024,1,2), datetime.date(2024,1,3), datetime.date(2024,1,4),
    datetime.date(2024,1,5), datetime.date(2024,1,6), datetime.date(2024,1,8),
    datetime.date(2024,1,9), datetime.date(2024,1,12), datetime.date(2024,1,13),
    datetime.date(2024,1,24), datetime.date(2024,1,25),
]

# 時間
class_time = {
    0: ['08:50:00', '10:30:00'],
    1: ['10:40:00', '12:20:00'],
    2: ['13:10:00', '14:50:00'],
    3: ['15:00:00', '16:40:00'],
    4: ['16:50:00', '18:30:00'],
    5: ['18:40:00', '20:20:00'],
    6: ['20:20:00', '22:00:00'],
    }

In [6]:
# カレンダーの作成
cal = Calendar()
cal.creator = 'Rintaro Fukui'

In [11]:
# 学期を判定
now_semester = None
today = datetime.date.today()
if (semester[0][0] <= today) & (today <= semester[0][1]):
    now_semester = 0
elif (semester[1][0] <= today) & (today <= semester[1][1]):
    now_semester = 1
elif (semester[2][0] <= today) & (today <= semester[2][1]):
    now_semester = 2
elif (semester[3][0] <= today) & (today <= semester[3][1]):
    now_semester = 3

now_semester

2

In [12]:
# 学期の開始日
start = semester[now_semester][0]

# 学期の終了日
stop = semester[now_semester][1]

while start <= stop:

    # 曜日を取得
    weekday = start.weekday()

    for i in range(len(jikanwari)):

        rele = jikanwari[i]

        # 日曜日ではない/授業がある/休日ではない を全て満たす場合にイベントを作成
        if weekday != 6 and rele[weekday] != '' and start not in holiday:

            # イベントを作成
            event = Event()

            # イベント名
            event.name = jikanwari[i][weekday]

            # 開始時刻
            event.begin = arrow.get(f"{start} {class_time[i][0]}", "YYYY-MM-DD HH:mm:ss").replace(tzinfo="Asia/Tokyo")

            # 終了時刻
            event.end = arrow.get(f"{start} {class_time[i][1]}" ,"YYYY-MM-DD HH:mm:ss").replace(tzinfo="Asia/Tokyo")

            # イベントをカレンダーに追加
            cal.events.add(event)

    start += datetime.timedelta(days=1)

In [13]:
str(cal)

/Users/rintarofukui/Private/Calendar-Generator/venv/lib/python3.10/site-packages/ics/component.py:85: FutureWarning: Behaviour of str(Component) will change in version 0.9 to only return a short description, NOT the ics representation. Use the explicit Component.serialize() to get the ics representation.
  warnings.warn(


'BEGIN:VCALENDAR\r\nVERSION:2.0\r\nPRODID:Rintaro Fukui\r\nBEGIN:VEVENT\r\nDTEND:20231019T093000Z\r\nDTSTART:20231019T075000Z\r\nSUMMARY:仏教（生き方を考える）発展[a11]\r\nUID:bfca9ab1-9c86-4580-bb65-2723a1be1af6@bfca.org\r\nEND:VEVENT\r\nBEGIN:VEVENT\r\nDTEND:20231102T093000Z\r\nDTSTART:20231102T075000Z\r\nSUMMARY:仏教（生き方を考える）発展[a11]\r\nUID:ad7ffba8-e273-4c5a-9b8c-531c95f8c3c1@ad7f.org\r\nEND:VEVENT\r\nBEGIN:VEVENT\r\nDTEND:20230927T055000Z\r\nDTSTART:20230927T041000Z\r\nSUMMARY:専門コース演習Ⅲ（ソーシャルイノベーション）\r\nUID:a423d59f-c04e-4af5-b0ee-e4e68b4e051f@a423.org\r\nEND:VEVENT\r\nBEGIN:VEVENT\r\nDTEND:20231031T013000Z\r\nDTSTART:20231030T235000Z\r\nSUMMARY:データマイニング\r\nUID:8ad6374b-dc03-45a2-a6a7-589ceeda6e73@8ad6.org\r\nEND:VEVENT\r\nBEGIN:VEVENT\r\nDTEND:20230925T074000Z\r\nDTSTART:20230925T060000Z\r\nSUMMARY:ビジネスモデル創出\r\nUID:3bcb08bf-39fd-497b-8f9a-e5fad57deede@3bcb.org\r\nEND:VEVENT\r\nBEGIN:VEVENT\r\nDTEND:20231002T074000Z\r\nDTSTART:20231002T060000Z\r\nSUMMARY:ビジネスモデル創出\r\nUID:f8f858e1-b289-49a4-8cc8-3c

In [ ]:
with open('my.ics', 'w', encoding='utf-8') as f:
    f.write(str(cal))